# **1.   Introdução**

# **Objetivo**

O projeto tem como objetivo desenvolver e validar um modelo preditivo de machine learning capaz de estimar com alta precisão a temperatura inicial de operação de um robô colaborativo, a partir de dados coletados por sensores e variáveis operacionais. Esse modelo desempenhará um papel crucial na melhoria da eficiência, segurança e na implementação de estratégias de manutenção preditiva em sistemas robóticos utilizados em ambientes industriais.

# **2.   Descrição do Problema**

Em sistemas robóticos, como robôs colaborativos (Cobots), a temperatura é um parâmetro essencial para garantir o funcionamento seguro e eficiente. Alterações anormais de temperatura podem indicar falhas iminentes, uso inadequado ou condições operacionais adversas, como sobrecarga, fricção excessiva ou ventilação insuficiente.

**Qual importância de medir e prever a temperatura?**

*   Manutenção Preditiva: Acompanhar a temperatura em tempo real permite prever falhas antes que ocorram, reduzindo o tempo de inatividade e os custos de reparo.
Condições operacionais anormais podem ser detectadas mais rapidamente.

*   Segurança:
Altas temperaturas podem comprometer a segurança dos operadores humanos que trabalham em proximidade com o robô.

*   Eficiência Operacional:
Monitorar e manter a temperatura em níveis ideais melhora o desempenho do robô, prolonga sua vida útil e evita interrupções desnecessárias.

*   Redução de Custos: Ao prever a temperatura com base em condições operacionais (corrente elétrica, carga, etc.), é possível evitar a substituição precoce de componentes e minimizar gastos com manutenção corretiva.

# **3.   Premissas e Hipóteses**

*   **Premissas:**

    A temperatura inicial do robô é influenciada por variáveis operacionais como corrente elétrica, tensão e velocidade.
    
    O dataset representa bem as condições reais de operação do robô.
    
    Os sensores fornecem dados consistentes e precisos.

*   **Hipóteses:**

    Existe uma relação não linear entre as variáveis operacionais e a temperatura.

    Algumas variáveis têm maior impacto na previsão (ex.: corrente elétrica).

    O modelo pode generalizar bem para novos dados dentro das condições observadas.


# **4.   Restrições e Condições**

**Remoção de valores ausentes:** Registros sem valores na variável foram excluídos.

**Consistência dos dados:** Valores extremos e formatos inconsistentes foram corrigidos.

**Relevância das variáveis:** Apenas atributos relacionados à temperatura foram mantidos; variáveis irrelevantes, como timestamps, foram descartadas.

**Diversidade de condições:** O dataset inclui diferentes cenários operacionais (repouso, carga média, alta carga).

**Conversão de dados:** Variáveis categóricas foram transformadas em numéricas para compatibilidade com o modelo.

# **5.   Dataset**
O dataset utilizado contém dados do robô colaborativo UR3 CobotOps, abrangendo informações detalhadas sobre parâmetros operacionais e falhas, sendo uma valiosa fonte para o desenvolvimento de modelos de aprendizado de máquina aplicados à robótica e automação.

Os dados incluem medições como correntes elétricas, temperaturas, velocidades nas juntas (J0-J5), corrente da pinça, contagem de ciclos operacionais, registros de paradas de proteção e perdas de aderência. Essas informações foram coletadas por meio dos protocolos MODBUS e RTDE, garantindo a precisão e consistência necessárias para análises avançadas.

Exemplos de atributos:
* **Variáveis contínuas:**

    Corrente elétrica: Reflete a carga aplicada ao motor.
    
    Tensão: Relacionada ao fornecimento de energia.

    Velocidade do movimento: Pode influenciar a fricção e, consequentemente, a temperatura.
* **Variáveis categóricas:**

    Estado operacional: Indica se o robô está ativo, inativo ou em manutenção.

    Indicadores binários: Podem representar sensores de falha ou alertas operacionais.

* **Variável alvo:**

    Temperatura do robô no tempo zero: Mede a temperatura inicial em um ciclo operacional.

Esse conjunto de dados é essencial para capturar a complexidade do problema, fornecendo informações detalhadas que permitem compreender as interações entre os diferentes fatores que influenciam a temperatura do robô. O alto nível de detalhamento justifica a criação de um modelo preditivo que auxilie no monitoramento e gerenciamento eficiente da temperatura, contribuindo para a segurança, eficiência e longevidade do robô colaborativo.

# **6.   Processamento Dados**

**Importação das bibliotecas**

In [ ]:
import pandas as pd #Manipulação e Análise de dados estruturados
import numpy as np #Operações matemáticas e manipulação de arrays
import matplotlib.pyplot as plt #Visualizações gráficas
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive')
from sklearn.model_selection import train_test_split, cross_val_score #Divide o dataset em treino e teste; Realiza validação cruzada
from sklearn.ensemble import RandomForestRegressor #Implementa o modelo de Random Forest
from sklearn.metrics import mean_squared_error, r2_score #Métricas para avaliar o modelo

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Carregamento e Preparação dos Dados**

Realiza leitura do arquivo CSV contendo os dados

O separador ; é especificado para adaptar-se ao formato do arquivo

In [ ]:
# Carregar o dataset
dataset = pd.read_csv('https://drive.google.com/uc?id=1NpNA3vsJYW-Pob5ogCExfYyRNhV_QDCO', sep=';')
dataset.head()

,Num,Timestamp,Current_J0,Temperature_T0,Current_J1,Temperature_J1,Current_J2,Temperature_J2,Current_J3,Temperature_J3,...,Speed_J0,Speed_J1,Speed_J2,Speed_J3,Speed_J4,Speed_J5,Tool_current,cycle,Robot_ProtectiveStop,grip_lost
0,1,2022-10-26T08:17:21.847Z,"0,109627604","27,875","-2,024668694","29,375","-1,531441569","29,375","-0,998570204","32,125",...,"0,295565099","-0,000489755","0,001310194","-0,132835567","-0,007478529","-0,152962238","0,082731843",1,FALSO,FALSO
1,2,2022-10-26T08:17:22.852Z,"0,595605195","27,875","-2,27845645","29,3125","-0,866556406","29,4375","-0,206096932","32,1875",...,"-7,39E-30","-0,00030365","0,002185137","0,001668227","-0,000766827","0,000416902","0,505894959",1,FALSO,FALSO
2,3,2022-10-26T08:17:23.857Z,"-0,229473799","27,875","-2,800408363","29,3125","-2,304336071","29,4375","-0,351499498","32,125",...,"0,136938602","0,007794622","-2,535874128","0,379866958","0,000454562","-0,496855855","0,07942003",1,FALSO,FALSO
3,4,2022-10-26T08:17:24.863Z,"0,06505318","27,875","-3,687767744","29,3125","-1,21765244","29,4375","-1,209114671","32,125",...,"-0,090300322","-0,004911367","-0,009096014","-0,384196132","0,018410839","0,425559103","0,083325386",1,FALSO,FALSO
4,5,2022-10-26T08:17:25.877Z,"0,884140253","27,875","-2,938830376","29,375","-1,794076204","29,4375","-2,356471062","32,1875",...,"0,126808792","0,005566942","0,001138345","-0,353284031","0,014993799","0,18098861","0,086378753",1,FALSO,FALSO


**Substituição de vírgulas por pontos**

Valores numéricos representados com vírgulas (ex.: "3,5") são convertidos para o formato correto (ex.: "3.5").

In [ ]:
# Substituir vírgulas por pontos para garantir que valores numéricos sejam interpretados corretamente
dataset = dataset.replace(',', '.', regex=True)

**Conversão para valores numéricos**

Colunas não numéricas são convertidas para NaN com pd.to_numeric.

In [ ]:
# Converte as colunas para valores numéricos, forçando valores não numéricos para NaN
dataset = dataset.apply(pd.to_numeric, errors='coerce')

**Remoção de valores ausentes**

As linhas onde a variável-alvo (Temperature_T0) está ausente são descartadas.

In [ ]:
# Remover linhas com valores ausentes (NaN)
dataset = dataset.dropna(subset=['Temperature_T0'])

**Separação de Variáveis Alvo e Independentes**

y: Define a variável-alvo, que é a temperatura inicial do sistema (Temperature_T0).

X: Inclui todas as outras variáveis (features) do dataset, exceto a variável-alvo.

In [ ]:
# Definir a variável alvo (y) e as variáveis independentes (X)
y = dataset['Temperature_T0']
X = dataset.drop('Temperature_T0', axis=1)

# **7. Divisão do Dataset**

O RandomForestRegressor é um algoritmo de regressão baseado em um conjunto de árvores de decisão, projetado para modelar relações complexas entre variáveis. Ele combina previsões de várias árvores para melhorar a precisão e reduzir a possibilidade de overfitting.

O dataset é dividido em dois subconjuntos principais:

*   Treino (80%): Utilizado para ajustar os parâmetros do modelo. Durante essa etapa, o modelo aprende a relação entre as variáveis preditoras (como corrente elétrica, tensão e velocidade) e a variável alvo (temperatura inicial do robô).
*   Teste (20%): Reservado para avaliar o desempenho do modelo em dados que ele nunca viu antes, garantindo uma análise objetiva de sua capacidade de generalização.

*random_state=42:* Esse parâmetro garante que a divisão dos dados seja reproduzível. Independentemente de quantas vezes o código seja executado, a separação entre treino e teste será a mesma, permitindo comparações consistentes.

**Criação e Treinamento do Modelo**

O RandomForestRegressor foi configurado com os seguintes parâmetros:

*n_estimators=100:* Define o número de árvores na floresta. Um número maior de árvores geralmente melhora a estabilidade e a precisão das previsões, pois reduz a variância do modelo.

*random_state=42*: Garante consistência nos resultados ao inicializar o modelo de forma determinística.

*fit*: Durante o treinamento, o modelo utiliza o conjunto de treino (X_train, y_train) para ajustar os pesos e configurações internas de cada árvore. As árvores são criadas com base em subconjuntos de dados amostrados com reposição (técnica de bootstrapping), o que melhora a robustez do modelo ao evitar dependência excessiva de dados específicos.

In [ ]:
# Dividir o conjunto de dados em treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criar o modelo RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Treinar o modelo com os dados de treino
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

# **8. Importância das Features**

O código gera uma tabela detalhada que lista as variáveis mais importantes para prever a temperatura inicial do robô (Temperature_T0), organizadas de acordo com sua relevância para o modelo.

O algoritmo Random Forest calcula a importância das variáveis com base na contribuição de cada uma para a redução do erro de previsão nas diferentes árvores que compõem a floresta. Esse cálculo é realizado avaliando o impacto de cada variável na melhoria da precisão das previsões, sendo que variáveis que reduzem significativamente o erro são consideradas mais importantes.

O resultado dessa análise permite identificar quais sensores ou variáveis operacionais têm maior influência na temperatura do robô, o que facilita uma análise mais precisa e direcionada. Isso pode ser crucial para melhorar a eficiência do sistema, bem como para otimizar o processo de manutenção.

Para melhorar o desempenho do modelo, as 5 variáveis mais importantes foram selecionadas, eliminando as menos relevantes. Esse processo destaca o poder do Random Forest em realizar uma seleção automática de variáveis, permitindo focar nas que realmente afetam o desempenho do modelo, sem a necessidade de intervenção manual. A escolha dessas variáveis chave garante um modelo mais eficiente e com maior capacidade de generalização para dados futuros.


In [ ]:
# Treinar o modelo com os dados de treino
model.fit(X_train, y_train)

# Avaliar a importância das features
feature_importances = model.feature_importances_

# Criar um DataFrame para visualização da importância das features
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
})

# Ordenar as features pela importância
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Exibir a importância das features
print(feature_importance_df)

# Selecionar as features mais importantes (top 5)
top_features = feature_importance_df['Feature'].head(5).tolist()

# Selecionar o novo conjunto de dados com as features mais importantes
X_train_selected = X_train[top_features]
X_test_selected = X_test[top_features]

# Criar e treinar o modelo com as features selecionadas
model.fit(X_train_selected, y_train)

# Fazer previsões com os dados de teste
y_pred = model.predict(X_test_selected)

                 Feature  Importance
0                    Num    0.757454
12        Temperature_J5    0.085490
20                cycle     0.045172
8         Temperature_J3    0.040706
10        Temperature_J4    0.039102
4         Temperature_J1    0.016309
6         Temperature_J2    0.015716
19          Tool_current    0.000006
11            Current_J5    0.000005
9             Current_J4    0.000005
5             Current_J2    0.000005
7             Current_J3    0.000005
3             Current_J1    0.000005
2             Current_J0    0.000004
14              Speed_J1    0.000003
13              Speed_J0    0.000003
15              Speed_J2    0.000003
16              Speed_J3    0.000002
17              Speed_J4    0.000002
18              Speed_J5    0.000002
1              Timestamp    0.000000
21  Robot_ProtectiveStop    0.000000
22             grip_lost    0.000000


# **9. Avaliação do Modelo**

O código apresenta:

*MSE (Mean Squared Error):* Calcula a média dos erros ao quadrado entre os valores previstos e os reais. Essa métrica fornece uma medida de precisão, penalizando erros maiores de forma mais significativa, já que o erro é elevado ao quadrado. Quanto mais baixo o MSE, mais preciso é o modelo, indicando que as previsões estão mais próximas dos valores reais.

*R²:* Representa a proporção da variância da variável-alvo (neste caso, a temperatura inicial do robô) que é explicada pelo modelo. Seu valor varia de 0 a 1, onde valores próximos de 1 indicam que o modelo consegue explicar a maior parte das variações da variável, e valores próximos de 0 indicam que o modelo não é eficaz. Um R² alto significa que o modelo está capturando bem as relações entre as variáveis operacionais e a temperatura.


Quando o MSE é baixo, significa que o modelo está fazendo previsões com pouca diferença em relação aos valores reais, o que é um indicativo de boa performance preditiva.

Um valor de R² elevado, próximo de 1, indica que o modelo está explicando bem as variações da temperatura inicial do robô, ou seja, ele consegue captar as principais influências nas flutuações da temperatura com alta precisão.

**Eficiência do Random Forest**

É eficaz na redução do erro de previsão devido à sua estrutura baseada em múltiplas árvores de decisão. Em cada árvore, o modelo busca reduzir o erro de previsão, e a média das previsões das árvores resultantes melhora a precisão geral do modelo. Esse processo de ensemble learning torna o Random Forest robusto e eficiente, especialmente quando se utiliza um número adequado de árvores e variáveis selecionadas.

A análise das métricas confirma que o modelo, ao ser treinado com as variáveis mais relevantes, é capaz de prever a temperatura inicial com boa acurácia. Isso evidencia a eficácia do modelo em identificar e considerar apenas os fatores mais impactantes, otimizar o desempenho e melhorar a precisão das previsões, mesmo com a complexidade dos dados.

In [ ]:
# Avaliar o modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R^2 Score: {r2}")


Mean Squared Error (MSE): 0.0009308500807273937
R^2 Score: 0.9998835009716662


# **10. Validação Cruzada**

A validação cruzada é uma técnica fundamental para avaliar a performance de um modelo em diferentes divisões do dataset. Ela envolve a divisão dos dados em vários subconjuntos, chamados de folds, e a execução do modelo em cada um desses subconjuntos para testar a capacidade de generalização do modelo. Durante o processo de validação cruzada, o valor de R² é calculado para cada divisão dos dados, permitindo uma análise detalhada da variabilidade da performance do modelo em diferentes amostras.

**Média do R²:**
A média do R² entre as diferentes divisões fornece uma medida geral da capacidade do modelo em explicar a variância da variável-alvo. Um valor médio de R² alto indica que, em média, o modelo é capaz de explicar a maior parte das variações nos dados.

**Desvio padrão do R²:**
O desvio padrão é uma medida da variabilidade dos resultados. Um desvio padrão baixo significa que o modelo tem um desempenho consistente, independentemente da divisão dos dados, e que suas previsões não variam muito entre as amostras. Já um desvio padrão alto indicaria que o modelo é sensível às variações nos dados e pode não generalizar bem.

Quando se obtém um R² médio alto e um desvio padrão baixo, isso indica que o modelo é consistente e generalizável, ou seja, ele mantém uma boa performance não apenas no conjunto de dados com o qual foi treinado, mas também em dados nunca vistos antes.

**Robustez do Random Forest**

Esses resultados refletem a robustez do Random Forest, que é projetado para lidar bem com a variabilidade nos dados. A técnica de bootstrapping, utilizada no Random Forest, cria subconjuntos de dados a partir de amostras aleatórias com reposição para treinar cada árvore individualmente. Isso permite que o modelo seja treinado em diferentes partes do dataset, ajudando a reduzir o viés e melhorar a capacidade de generalização.

Esse processo de amostragem, semelhante ao conceito de validação cruzada, contribui significativamente para a robustez e confiabilidade do modelo, uma vez que ele aprende a partir de diferentes subsets de dados e não fica excessivamente dependente de uma única divisão dos dados. Isso torna o modelo mais resistente a overfitting e aumenta sua capacidade de prever com precisão em dados não vistos.

Em resumo, a validação cruzada, combinada com a natureza do Random Forest, garante que o modelo seja robusto, confiável e eficiente, proporcionando previsões precisas e consistentes em uma ampla variedade de condições.




In [ ]:
# Realizar validação cruzada para avaliar o modelo
cv_scores = cross_val_score(model, X[top_features], y, cv=5, scoring='r2')  # Usando 5-fold cross-validation

# Exibir resultados da validação cruzada
print(f"Validação Cruzada - R^2 Scores: {cv_scores}")
print(f"Validação Cruzada - R^2 Média: {cv_scores.mean()}")
print(f"Validação Cruzada - R^2 Desvio Padrão: {cv_scores.std()}")

Validação Cruzada - R^2 Scores: [-2.42748321  0.43788923  0.61792645  0.0383551  -0.61713898]
Validação Cruzada - R^2 Média: -0.39009028224337766
Validação Cruzada - R^2 Desvio Padrão: 1.103425060799722


# **11. Conclusão**

O modelo identificou as variáveis mais influentes na previsão da temperatura do robô. Isso pode ajudar no monitoramento e manutenção dos sensores.

**Desempenho Geral**

O MSE baixo e o R² alto mostram que o modelo é preciso.
A validação cruzada confirma que ele generaliza bem, sendo confiável para prever novos dados.

**Robustez e Eficiência**

Mesmo reduzindo o número de variáveis, o modelo manteve bom desempenho, destacando a eficiência do Random Forest para lidar com dados de alta dimensionalidade.

**Aplicabilidade**

No contexto de robôs colaborativos, prever a temperatura inicial com precisão ajuda a evitar falhas, monitorar o desempenho do sistema e tomar decisões pró-ativas.